In [34]:
%pip install networkx plotly ipywidgets pandas anywidget nbformat>=4.2.0


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /home/pienskoi/PycharmProjects/TRSKS/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import networkx as nx
import plotly.graph_objects as go
from ipywidgets import widgets, VBox

CONFIG = {
    "su_count": 4,
    "spines_per_su": 4,
    "leaves_per_su": 8,
    "nodes_per_su": 32,
}

CONFIG['spine_count'] = CONFIG['spines_per_su'] * CONFIG['su_count']

def create_graph():
    G = nx.Graph()
    pos = {}
    node_props = {}

    for s_idx in range(1, CONFIG['spine_count'] + 1):
        spine_name = f"Spine-{s_idx:02d}"
        G.add_node(spine_name, layer='spine')

        pos[spine_name] = (s_idx * 5, 3)

        node_props[spine_name] = {'color': '#000000', 'size': 20, 'label': spine_name}

    for su_idx in range(1, CONFIG['su_count'] + 1):
        su_x_offset = (su_idx - 1) * 22

        for l_idx in range(1, CONFIG['leaves_per_su'] + 1):
            leaf_name = f"Leaf-{su_idx:02d}-{l_idx:02d}"
            G.add_node(leaf_name, layer='leaf')

            pos[leaf_name] = (su_x_offset + (l_idx * 2.5), 2)

            node_props[leaf_name] = {'color': '#333333', 'size': 15, 'label': leaf_name}

            for i in range(1, CONFIG['spine_count'] + 1):
                spine_target = f"Spine-{i:02d}"
                G.add_edge(leaf_name, spine_target, color='#d62728', width=2)

    node_global_idx = 1
    for su_idx in range(1, CONFIG['su_count'] + 1):
        su_x_offset = (su_idx - 1) * 22

        for n_idx in range(1, CONFIG['nodes_per_su'] + 1):
            node_name = f"H100-{node_global_idx:03d}"
            G.add_node(node_name, layer='compute')

            pos[node_name] = (su_x_offset + (n_idx * 0.7) - 1, 1)

            node_props[node_name] = {
                'color': '#8c735a', 'size': 12, 'label': node_name}

            for r_idx in range(1, CONFIG['leaves_per_su'] + 1):
                leaf_target = f"Leaf-{su_idx:02d}-{r_idx:02d}"
                G.add_edge(node_name, leaf_target, color='#ff7f0e', width=1)

            node_global_idx += 1

    return G, pos, node_props

def plot_graph(G, pos, node_props):
    fig = go.Figure()

    for u, v, data in G.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        fig.add_trace(go.Scatter(
            x=[x0, x1], y=[y0, y1],
            mode='lines',
            line=dict(color=data['color'], width=data['width']),
            hoverinfo='none', showlegend=False, opacity=0.6
        ))

    nodes_by_style = {}
    for node, props in node_props.items():
        key = (props['color'], props['size'])
        if key not in nodes_by_style:
            nodes_by_style[key] = {'x': [], 'y': [], 'text': []}

        nodes_by_style[key]['x'].append(pos[node][0])
        nodes_by_style[key]['y'].append(pos[node][1])
        nodes_by_style[key]['text'].append(props['label'])

    for (color, size), data in nodes_by_style.items():
        plotly_shape = 'square'

        fig.add_trace(go.Scatter(
            x=data['x'], y=data['y'],
            mode='markers',
            text=data['text'],
            hoverinfo='text',
            marker=dict(symbol=plotly_shape, color=color, size=size, line=dict(width=1, color='black')),
            showlegend=False
        ))

    fig.update_layout(
        title="Eos NVIDIA DGX SuperPOD Topology",
        template='plotly_white',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
        height=600, width=1200,
        margin=dict(b=10,l=10,r=10,t=40),
        hovermode='closest'
    )

    fig.show()

G, pos, node_props = create_graph()
plot_graph(G, pos, node_props)